<a href="https://colab.research.google.com/github/Riazhatvi/riazkhan1/blob/main/NLP_project_of_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk as tk
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
tk.download('punkit')

[nltk_data] Error loading punkit: Package 'punkit' not found in index


False

In [ ]:
data_set = ("/content/drive/MyDrive/NLP_Project/topical_chat.csv")

In [ ]:
data_set = pd.read_csv("/content/drive/MyDrive/NLP_Project/topical_chat.csv")

In [ ]:
data_set

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [ ]:
data_set['message'] = data_set['message'].apply(lambda x: x.lower())
# data_set['message'] = data_set['message'].split(" ")

In [ ]:
data_set['message']

0                     are you a fan of google or microsoft?
1          both are excellent technology they are helpfu...
2          i'm not  a huge fan of google, but i use it a...
3          google provides online related services and p...
4          yeah, their services are good. i'm just not a...
                                ...                        
188373     wow, it does not seem like that long. since i...
188374     i havent seen that episode, i might google it...
188375     i don't think i have either. that's an insane...
188376     i did, my little brother used to love thomas ...
188377     it did. ringo starr, george carlin, and alec ...
Name: message, Length: 188378, dtype: object

In [ ]:
questions = data_set['message']
answers = data_set['message']


In [ ]:
x= []
y=[]
for i in range (len(data_set)-1):
  question_text = data_set['message'].iloc[i]
  answer_text = data_set['message'].iloc[i]
  x.append(question_text)
  y.append(answer_text)


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

x_tokenized = [word_tokenize(question_text) for question_text in x]
y_tokenized = [word_tokenize(answer_text) for answer_text in y]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Data Splitting:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_tokenized, y_tokenized, test_size=0.2, random_state=42)


In [ ]:
# Vocabulary Creation:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train + y_train)

vocab_size = len(tokenizer.word_index) + 1


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

def clean_text(text):

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train + y_train)

x_train_encoded = tokenizer.texts_to_sequences(x_train)
y_train_encoded = tokenizer.texts_to_sequences(y_train)

x_test_encoded = tokenizer.texts_to_sequences(x_test)
y_test_encoded = tokenizer.texts_to_sequences(y_test)


In [ ]:
max_sequence_length = 100
x_train_padded = pad_sequences(x_train_encoded, maxlen=max_sequence_length, padding='post', truncating='post')
y_train_padded = pad_sequences(y_train_encoded, maxlen=max_sequence_length, padding='post', truncating='post')

x_test_padded = pad_sequences(x_test_encoded, maxlen=max_sequence_length, padding='post', truncating='post')
y_test_padded = pad_sequences(y_test_encoded, maxlen=max_sequence_length, padding='post', truncating='post')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model
output_units = vocab_size
embedding_dim = 100  # You can adjust this to your desired embedding dimension
max_sequence_length = 100  # Replace with your desired sequence length

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(output_units, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

num_epochs = 10
batch_size = 32

model.fit(x_train_padded, y_train_padded, epochs=num_epochs, batch_size=batch_size, validation_data=(x_test_padded, y_test_padded))


loss, accuracy = model.evaluate(x_test_padded, y_test_padded)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
predictions = model.predict(x_new_data_padded)
model.save("your_model.h5")
loaded_model = tf.keras.models.load_model("your_model.h5")


Epoch 1/10


ValueError: ignored